<a href="https://colab.research.google.com/github/Retr0Sys/Clasificaci-n-de-Rumores-en-Redes-Sociales-CFE-IA/blob/main/Clasificaci%C3%B3n_de_Rumores_en_Redes_Sociales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importamos el .csv**

In [85]:
import pandas as pd
df = pd.read_csv("dataset_train_val.csv")

## **Vemos parte del contenido con un head**

In [86]:
df.head(5)

,text,is_rumor,user.handle,topic
0,@korikisulda listen. Hey are protesting agains...,0.0,vida_andras,charliehebdo
1,“@TheEyeOfControl: Americans are 8 times more ...,1.0,SoulBrothaIII,ferguson
2,@Israel @IsraelMFA #Itsallaboutisrael (We kil...,0.0,mdp4202,NaN
3,@matwilson6 @capetownbrown Always do.,0.0,scottsmith349,ferguson
4,"@TomGauthey I'm out of date already of course,...",0.0,CMHarkey,charliehebdo


## **Comprobamos si hay datos vacios**

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59322 entries, 0 to 59321
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   text         59322 non-null  object 
 1   is_rumor     59320 non-null  float64
 2   user.handle  59320 non-null  object 
 3   topic        47196 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.8+ MB


## **Vemos la cantidad de datos vacíos**

In [88]:
df.isnull().sum()

,0
text,0
is_rumor,2
user.handle,2
topic,12126


## **Hacemos limpieza y adaptación de los datos**

In [89]:
df.drop(columns='topic', inplace=True)
df['is_rumor'].fillna(0, inplace=True)
df['user.handle'].fillna('unknown', inplace=True)
df.isnull().sum()

/tmp/ipython-input-2298042494.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['is_rumor'].fillna(0, inplace=True)
/tmp/ipython-input-2298042494.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me

,0
text,0
is_rumor,0
user.handle,0


## **Eliminamos duplicados**

In [90]:
print("Cantidad de filas antes de eliminar duplicado", df.shape)
df.drop_duplicates(inplace=True)
print("Cantidad de filas después de eliminar duplicado", df.shape)

Cantidad de filas antes de eliminar duplicado (59322, 3)
Cantidad de filas después de eliminar duplicado (58606, 3)


## **Creamos el modelo**

In [91]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['is_rumor'], test_size=0.25)

In [92]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizador = CountVectorizer()
X_train_vec = vectorizador.fit_transform(X_train)
X_test_vec = vectorizador.transform(X_test)

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

modelo = LogisticRegression()
modelo.fit(X_train_vec, y_train)

predit = modelo.predict(X_test_vec)

accuracy = accuracy_score(y_test, predit)
print("Porcentaje de acierto:", accuracy * 100)

Porcentaje de acierto: 90.47911547911548


## **Leemos el .csv de prueba**

In [94]:
test_data = pd.read_csv('dataset_test_sin_etiqueta.csv')

test_data.head()

,text,user.handle,topic,ID
0,@WSJ haven't we heard that about four times al...,L4rryC,NaN,1
1,"@bopanc @FT @LukeReuters well, there goes any ...",freethink120,charliehebdo,2
2,@iowahawkblog @RamziHabre You mean that wasn't...,kshaidle,charliehebdo,3
3,@OmarWaraich @imsabbah #BreakingNews Correspon...,ShafeekTg,charliehebdo,4
4,@_vaNessasary not the point I was trying to ma...,NBully18,ferguson,5


In [95]:
test_data.drop(columns='topic', inplace=True)
test_data['user.handle'].fillna('unknown', inplace=True)

/tmp/ipython-input-2221952409.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data['user.handle'].fillna('unknown', inplace=True)


## **Creamos el archivo de salida**

In [96]:
X_test_vec = vectorizador.transform(test_data['text'])

predict = modelo.predict(X_test_vec)

output = pd.DataFrame({'ID': test_data['ID'], 'etiqueta': predict})
output.to_csv('submission.csv', index=False)

print("Archivo creado con éxito")

Archivo creado con éxito
